<a href="https://colab.research.google.com/github/monicafar147/classification-predict-streamlit-template/blob/Modeling/trying_various_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install comet_ml

     |████████████████████████████████| 215kB 4.7MB/s 
     |████████████████████████████████| 204kB 13.6MB/s 
     |████████████████████████████████| 419kB 11.2MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34546 sha256=7404134c1341239f353a9083d3227478ab0225629bfb06f835c0f1694a1fb514
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj


In [2]:
# import comet_ml in the top of your file
from comet_ml import Experiment
    
# Add the following code anywhere in your machine learning file
experiment = Experiment(api_key="rBqQ3hDuEa6xVpT9ns5Tz1dVt",project_name="nlp-climate-change", workspace="monicafar147")

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/monicafar147/nlp-climate-change/39a80c700148442781be1ed3cae130fe



In [ ]:
!pip install contractions

In [ ]:
import numpy as np 
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-deep')

# text preprocessing
import re
import string
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import Word
from wordcloud import WordCloud, STOPWORDS

# models
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
train = pd.read_csv("https://raw.githubusercontent.com/monicafar147/classification-predict-streamlit-template/master/climate-change-belief-analysis/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/monicafar147/classification-predict-streamlit-template/master/climate-change-belief-analysis/test.csv")

In [ ]:
print("Train\n")
print(train.head(5))
print("\nTest")
print(test.head(5))

Train

   sentiment                                            message  tweetid
0          1  PolySciMajor EPA chief doesn't think carbon di...   625221
1          1  It's not like we lack evidence of anthropogeni...   126103
2          2  RT @RawStory: Researchers say we have three ye...   698562
3          1  #TodayinMaker# WIRED : 2016 was a pivotal year...   573736
4          1  RT @SoyNovioDeTodas: It's 2016, and a racist, ...   466954

Test
                                             message  tweetid
0  Europe will now be looking to China to make su...   169760
1  Combine this with the polling of staffers re c...    35326
2  The scary, unimpeachable evidence that climate...   224985
3  @Karoli @morgfair @OsborneInk @dailykos \nPuti...   476263
4  RT @FakeWillMoore: 'Female orgasms cause globa...   872928


In [ ]:
def _preprocess(data):
  df = data.copy()

  # apply lowercase to data
  data['message'] = data['message'].apply(lambda word: ''.join(word.lower()))

  # function to remove contraction
  def remove_contraction(row):
    fixed = [contractions.fix(word) for word in row.split()]
    return ' '.join(map(str,fixed))

  # replace contractions
  df['message'] = np.vectorize(remove_contraction)(df['message'])

  # function to remove patterns
  def remove_pattern(text,pattern,replacement=''):
    remove_this = re.findall(pattern, text)
    for item in remove_this:
      text = re.sub(item, replacement, text)
    return text

  # remove URL
  df['message'] = df['message'].apply(lambda word: re.split('https:\/\/.*', str(word))[0])

  # remove punctuation
  df['message'] = df['message'].apply(lambda word: word.translate(str.maketrans('', '', string.punctuation)))

  # remove stopwords
  stop_words = stopwords.words('english')
  data['message'] = data['message'].apply(lambda word: ' '.join(word for word in word.split() if word not in stop_words))

  # remove retweet as rt
  df['message'] = np.vectorize(remove_pattern)(df['message'],"RT[\w]*")
  return df

In [ ]:
trained = train[['sentiment','message','tweetid']]
tested = test[['message','tweetid']]

In [ ]:
train_processed = _preprocess(trained)
test_processed = _preprocess(tested)

In [ ]:
from sklearn.model_selection import train_test_split
# Splitting the labels and features
X = train_processed['message']
y = train_processed['sentiment']

# Splitting the labels and fetures into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=42)

In [ ]:
# Decision tree
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import tree

clf = tree.DecisionTreeClassifier()
text_cls = Pipeline([('tfidf',TfidfVectorizer()),('classify',clf)])
text_cls.fit(X_train, y_train)

pred = text_cls.predict(X_test)
from sklearn.metrics import confusion_matrix,classification_report
report_1 = print(classification_report(y_test, pred))
report_1

              precision    recall  f1-score   support

          -1       0.36      0.27      0.31       126
           0       0.34      0.33      0.33       224
           1       0.72      0.72      0.72       895
           2       0.58      0.67      0.62       337

    accuracy                           0.62      1582
   macro avg       0.50      0.50      0.50      1582
weighted avg       0.61      0.62      0.61      1582



In [ ]:
# Linear SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

svc = LinearSVC()
text_svc = Pipeline([('tfidf',TfidfVectorizer()),('classify',svc)])
text_svc.fit(X_train, y_train)

pred_2 = text_svc.predict(X_test)
from sklearn.metrics import confusion_matrix,classification_report
report_1 = print(classification_report(y_test, pred_2))
report_1

              precision    recall  f1-score   support

          -1       0.64      0.49      0.56       126
           0       0.58      0.35      0.44       224
           1       0.79      0.86      0.82       895
           2       0.72      0.78      0.75       337

    accuracy                           0.74      1582
   macro avg       0.68      0.62      0.64      1582
weighted avg       0.73      0.74      0.73      1582



In [ ]:
# Gaussian not working though
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import todense
naive_bayes = GaussianNB()
text_nb = Pipeline([('tfidf',TfidfVectorizer()),('classify',naive_bayes)])
XD_train = X_train.todense()
text_nb.fit(XD_train, y_train)

pred_3 = text_nb.predict(X_test)
report_1 = print(classification_report(y_test, pred_3))
report_1


ModuleNotFoundError: ignored

In [ ]:
# K nearest neighbours, 250 seems best but maybe could be a better option
n_neighbors = 250 
knn = KNeighborsClassifier(n_neighbors)
text_knn = Pipeline([('tfidf',TfidfVectorizer()),('classify',knn)])
text_knn.fit(X_train, y_train)

pred_4 = text_knn.predict(X_test)
report_1 = print(classification_report(y_test, pred_4))
report_1



              precision    recall  f1-score   support

          -1       1.00      0.01      0.02       126
           0       0.88      0.03      0.06       224
           1       0.63      0.97      0.76       895
           2       0.81      0.44      0.57       337

    accuracy                           0.65      1582
   macro avg       0.83      0.36      0.35      1582
weighted avg       0.73      0.65      0.56      1582



In [ ]:
# MulitnomialNB
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
text_mnb = Pipeline([('tfidf',TfidfVectorizer()),('classify',model)])
text_mnb.fit(X_train, y_train)
pred_5 = text_mnb.predict(X_test)
report_5 = print(classification_report(y_test, pred_5))
report_5


              precision    recall  f1-score   support

          -1       1.00      0.01      0.02       126
           0       1.00      0.03      0.06       224
           1       0.62      0.99      0.76       895
           2       0.90      0.42      0.57       337

    accuracy                           0.65      1582
   macro avg       0.88      0.36      0.35      1582
weighted avg       0.77      0.65      0.56      1582



In [ ]:
# Logistic regression
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
text_lr = Pipeline([('tfidf',TfidfVectorizer()),('classify',lr)])
text_lr.fit(X_train, y_train)
pred_6 = text_lr.predict(X_test)
report_6 = print(classification_report(y_test, pred_6))
report_6

              precision    recall  f1-score   support

          -1       0.79      0.36      0.49       126
           0       0.64      0.31      0.42       224
           1       0.75      0.91      0.82       895
           2       0.76      0.75      0.76       337

    accuracy                           0.75      1582
   macro avg       0.74      0.58      0.62      1582
weighted avg       0.74      0.75      0.73      1582



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
